In [3]:
%load_ext autoreload
%autoreload 2
from dataset import ManualFeatureDataModule
from pathlib import Path
import torch

In [4]:
root_dir = Path("/media/nvme1/icare-data/6h-combined/")
labels_dir = Path("/home/bc299/icare/artifacts/patient_data.csv")
dm = ManualFeatureDataModule(root_dir=root_dir, labels_csv=labels_dir)
dm.setup()

In [12]:
for batch_idx, (x, y) in enumerate(dm.train_dataloader()):
    print(x.size())

torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([32, 14, 144])
torch.Size([